To-Do:
- Import WMT dataset and perform actual training.

Import packages

In [1]:
import torch
from torch import nn

import spacy

import math

/home/achalhoub/miniconda3/envs/transformer_pt/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load English and German tokenizers using Spacy.

In [ ]:
eng_lang = spacy.load("en_core_web_sm")
ger_lang = spacy.load("de_core_news_sm")

Specify device as GPU

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


Create a single batch of random input just to use to check if the shape initializations in the below classes are correct.

In [12]:
torch.cuda.empty_cache()

Set hyperparameters. I used the same ones as those used in the paper.

In [13]:
num_heads = 8
embed_len = 512
seq_len = 20                # just a dummy number for now
batch_size = 8              # just a dummy number for now
stack_len = 6               # length of encoder and decoder stacks (=6 as used in paper)
output_vocab_size = 4000    # just a dummy number for now
dropout = 0.1
input_vocab_size = 37000    # just a dummy number for now (from paper)

In [ ]:
test_input = torch.rand((batch_size, seq_len, embed_len)).to(device)

Create Input embedding class. This includes both the normal embedding for the tokenized input sequences, as well as the positional embeddings. They are added together and the sum of them is used as the input embedding to the encoder.

In [39]:
class InputEmbedding(nn.Module):
    def __init__(self, input_vocab_size=input_vocab_size, embed_len=embed_len, dropout=dropout, device=device):
        super(InputEmbedding, self).__init__()
        self.input_vocab_size = input_vocab_size
        self.embed_len = embed_len
        self.device = device
        self.dropout = dropout

        self.firstEmbedding = nn.Embedding(self.input_vocab_size, self.embed_len)
        self.secondEmbedding = nn.Embedding(self.input_vocab_size, self.embed_len)

        self.dropoutLayer = nn.Dropout(p=self.dropout)

    def forward(self, input):
        first_embedding = self.firstEmbedding(input)
        
        batch_size, seq_len = input.shape

        positions_vector = torch.arange(0, seq_len).expand(batch_size, seq_len).to(self.device)
        second_embedding = self.secondEmbedding(positions_vector)

        return self.dropoutLayer(first_embedding + second_embedding)

In [40]:
input_tokens = torch.randint(10, (8, 20)).to(device)
input_tokens.get_device()

0

In [45]:
test_embedding = InputEmbedding().to(device)
print(test_embedding.forward(input_tokens))

first embedding size:  torch.Size([8, 20, 512])
second embedding size:  torch.Size([8, 20, 512])
first embedding:  tensor([[[-0.6014,  0.8802, -1.0483,  ..., -0.4807, -0.3084,  0.0236],
         [-0.1844,  1.3597, -2.2082,  ..., -0.5663,  0.6736, -0.9453],
         [-0.6014,  0.8802, -1.0483,  ..., -0.4807, -0.3084,  0.0236],
         ...,
         [-0.1844,  1.3597, -2.2082,  ..., -0.5663,  0.6736, -0.9453],
         [-1.4538, -1.0109, -0.4769,  ...,  0.3788, -0.5659, -0.6634],
         [-0.5754, -1.5874,  1.5591,  ...,  0.2740,  1.0448, -0.2669]],

        [[-1.4602, -0.9560,  0.1756,  ...,  1.0818,  0.2341,  0.1611],
         [ 0.5136, -2.6123, -0.2758,  ..., -1.7128,  0.4511, -1.0418],
         [-1.3922, -0.1399,  0.3916,  ..., -0.9458,  1.0900,  1.4927],
         ...,
         [-0.6418, -0.7586, -1.7455,  ..., -0.3027,  0.5802,  0.4377],
         [-0.3961, -1.1131, -0.4194,  ...,  0.0183, -0.4295, -0.8628],
         [ 0.5136, -2.6123, -0.2758,  ..., -1.7128,  0.4511, -1.0418]],

 

Begin building the Transformer. The first step is to build the 'Scaled Dot-Product Attention' block mentioned in the paper.

In [ ]:
class ScaledDotProduct(nn.Module):
    def __init__(self, embed_len=embed_len, mask=None):
        super(ScaledDotProduct, self).__init__()
        
        self.dk = embed_len                 # dk = embed_len
        self.mask = mask
        self.softmax = nn.Softmax(dim=2)    # Softmax operator

    # Define the forward function
    def forward(self, queries, keys, values):       
        compatibility = torch.bmm(queries, torch.transpose(keys, 1, 2))    # first batch MatMul operation
        compatibility = compatibility / math.sqrt((self.dk))               # scaling down by sqrt(dk)  -> (8, 20, 20)

        # Apply mask after scaling the result of MatMul of Q and K.
        # This is needed in the decoder to prevent the decoder from
        # 'peaking ahead' and knowing what word will come next.
        # Check: https://pytorch.org/docs/stable/generated/torch.tril.html 
        if self.mask is not None:
            compatibility = torch.tril(compatibility)
            
        compatibility_softmax = self.softmax(compatibility)                # normalizing using Softmax -> (8, 20, 512) 
        return torch.bmm(compatibility_softmax, values)                    # final batch MatMul operation

Build the 'Multi-Head Attention' block. Init variable need simplifying.

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads=num_heads, embed_len=embed_len, batch_size=batch_size, seq_len=seq_len, mask=None):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.embed_len = embed_len
        self.head_length = self.embed_len/self.num_heads
        self.mask = mask
        self.concat_output = []

        # For an input, Q for example, which would originally have a shape
        # of (N, seq_len, embed_len), it would be split up into the number of 
        # heads that we define (ex: 8). So, the new shape would be
        # (N, seq_len, embed_len/8). This would also apply to K and V too.

        # Since we are flattening batches of matrices, I'm not sure if the flattening
        # should be done in another way. I'll come back to this later if it needs changing.
        self.q_in = self.k_in = self.v_in = self.seq_len * self.embed_len       # 20*512 = 10240

        # For the input of each Linear layer, we would have the divided Q, K, 
        # and V calculated above. q_in, k_in, and v_in = 10240 each. So each
        # linear will have input size = output size = 10240/8 = 1280.
        self.q_linear = nn.Linear(int(self.q_in/8), int(self.q_in/8))
        self.k_linear = nn.Linear(int(self.k_in/8), int(self.k_in/8))
        self.v_linear = nn.Linear(int(self.v_in/8), int(self.v_in/8))

        # Attention layer.
        if self.mask is not None:
            self.attention = ScaledDotProduct(mask=True) 
        else:
            self.attention = ScaledDotProduct()

        # This is the final Linear layer, after the outputs of all the heads
        # from the Scaled Dot Product layer have been concatenated together. The
        # output dimension of this layer is a hyperparameter that we define. Here
        # we use embed_len, which is 512.
        self.output_linear = nn.Linear(self.q_in, self.embed_len)

    def forward(self, queries, keys, values):
        # Feed the 8 heads of Q, K, and V into the linear layers in parallel, and then into the
        # attention block. Let's say the original tensor Q has the following shape: 
        # (N, seq_len, embed_len) -> (8, 20, 512).
        # The segment that will go into each head will be of the following size:
        # (N, seq_len, embed_len/num_heads) -> (8, 20, 64). So we need to slice the third dimension.
        for i in range(self.num_heads):

            # The output of each of the linear layers has length -> (N, seq_len*embed_len/num_heads) -> (N, 1280)
            q_linear_output = self.q_linear(torch.flatten(queries[:, :, int(i*self.head_length):int((i+1)*self.head_length)], start_dim=1, end_dim=2))
            k_linear_output = self.k_linear(torch.flatten(keys[:, :, int(i*self.head_length):int((i+1)*self.head_length)], start_dim=1, end_dim=2))
            v_linear_output = self.v_linear(torch.flatten(values[:, :, int(i*self.head_length):int((i+1)*self.head_length)], start_dim=1, end_dim=2))

            # Since the three outputs computed from the linear layers above are just 1D vectors of length
            # (N, seq_len*embed_len/num_heads) -> (N, 1280), and the ScaledDotProduct forward method expects 3D tensors,
            # I will reshape the 1D vectors into 3D tensors of shape (N, seq_len, embed_len/num_heads)
            q_reshaped_output = torch.reshape(q_linear_output, (self.batch_size, self.seq_len, int(self.embed_len/self.num_heads)))
            k_reshaped_output = torch.reshape(k_linear_output, (self.batch_size, self.seq_len, int(self.embed_len/self.num_heads)))
            v_reshaped_output = torch.reshape(v_linear_output, (self.batch_size, self.seq_len, int(self.embed_len/self.num_heads)))

            #print('q_reshaped_output shape: ', q_reshaped_output.shape)

            # Feed reshaped Q, K, and V into ScaledDotProduct layer.
            # 'sdp_output' should have shape (N, seq_len, embed_len/num_heads)
            sdp_output = self.attention.forward(q_reshaped_output, k_reshaped_output, v_reshaped_output)

            # Each 'sdp_output' is a Tensor of shape (N, seq_len, embed_len/num_heads) -> (8, 20, 64).
            # Each flattened Tensor has length (8, 20*64) = 10240
            #print('sdp_output shape: ', sdp_output.shape)
            #print('sdp_output flattened length: ', torch.flatten(sdp_output, start_dim=1, end_dim=2).shape)
            
            # We need to concatenate the outputs of all the heads
            # into one vector and pass it through a final linear layer
            self.concat_output.append(torch.flatten(sdp_output, start_dim=1, end_dim=2))
            
        flattened_concat_output = torch.flatten(torch.stack(self.concat_output), start_dim=1, end_dim=2)
        
        # Pass the concatenated vector in a final linear layer and return output
        return self.output_linear(flattened_concat_output)

Test shapes

In [ ]:
multihead = MultiHeadAttention(mask=True).to(device)

In [ ]:
test_output = multihead.forward(test_input, test_input, test_input)

In [ ]:
print(test_output.size())

Building the Encoder block. I will then stack this into multiple layers to create the Encoder stack.

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, embed_len=embed_len, dropout=dropout):
        super(EncoderBlock, self).__init__()

        self.embed_len = embed_len
        self.dropout = dropout
        self.multihead = MultiHeadAttention()             # Multi-Head Attention layer
        self.firstNorm = nn.LayerNorm(embed_len)          # Normalization layer (after the multi-head attention layer)
        self.secondNorm = nn.LayerNorm(embed_len)         # Normalization layer (after the Feed Forward layer)
        self.dropoutLayer = nn.Dropout(p=self.dropout)    # Dropout layer (before addition and normalization)

        # The Feed Forward layer. In the paper this has input &
        # output = 512 (or = embed_len) and inner-layer = 2048 (or = embed_len*4)
        self.feedForward = nn.Sequential(
            nn.Linear(embed_len, embed_len*4),
            nn.ReLU(),
            nn.Linear(embed_len*4, embed_len)
        )

    def forward(self, queries, keys, values):
        attention_output = self.multihead.forward(queries, keys, values)
        attention_output = self.dropoutLayer(attention_output)

        # the output of the first residual connection
        first_sublayer_output = self.firstNorm(attention_output + queries)

        ff_output = self.feedForward(first_sublayer_output)
        ff_output = self.dropoutLayer(ff_output)

        # return the output of the second residual connection
        return self.secondNorm(ff_output + first_sublayer_output)

Building the Decoder block. I will also stack this into multiple layers to create the Decoder stack.

The decoder has a total of 3 inputs: the queries, which come from the previous decoder layer, and the memory keys and values, which come from the output of the encoder (Section 3.2.3).

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_len=embed_len, dropout=dropout):
        super(DecoderBlock, self).__init__()

        self.embed_len = embed_len
        self.dropout = dropout

        # Masked Multi-Head Attention and Normalization layers.
        self.maskedMultihead = MultiHeadAttention(mask=True)
        self.firstNorm = nn.LayerNorm(self.embed_len)

        self.dropoutLayer = nn.Dropout(p=self.dropout)

        # The output of the above two layers and the output from the encoder stack feed 
        # into an 'encoder block'
        self.encoderBlock = EncoderBlock()

    def forward(self, queries, keys, values):

        # First sublayer, which consists of the Masked Multi-Head Attention + Normalization
        # sublayer, with a residual connection
        masked_multihead_output = self.maskedMultihead.forward(queries, queries, queries)
        masked_multihead_output = self.dropoutLayer(masked_multihead_output)
        first_sublayer_output = self.firstNorm(masked_multihead_output + queries)

        # The remaining of the DecoderBlock is basically an encoder block, which takes keys 
        # and values from the actual Encoder stack output, and takes queries from the 
        # previous sublayer of the DecoderBlock
        return self.encoderBlock.forward(first_sublayer_output, keys, values)      

Create whole Transformer block (still missing some components).

In [ ]:
class Transformer(nn.Module):
    def __init__(self, stack_len=stack_len, embed_len=embed_len, output_vocab_size=output_vocab_size):
        super(Transformer, self).__init__()
        self.stack_len = stack_len
        self.embed_len = embed_len
        self.output_vocab_size = output_vocab_size

        self.encStack = nn.ModuleList([EncoderBlock() for i in range(self.stack_len)])
        self.decStack = nn.ModuleList([DecoderBlock(mask=True) for i in range(self.stack_len)])
        self.finalLinear = nn.Linear(self.embed_len, self.output_vocab_size)
        self.softmax = nn.Softmax()

    def forward(self, test_input):
        enc_output = test_input

        # Final output 'enc_output' of this loop will be both the key and value
        # that will be taken as input to the second sub-layer of the decoder
        for enc_layer in self.encStack:
            enc_output = enc_layer.forward(enc_output, enc_output, enc_output)

        # Decoder stack will take the 'enc_output' from the decoder as the keys
        # and values, and will take its own output from the previous layer as
        # the query. The query used for the first layer is the '<sos>' token.
        for dec_layer in self.decStack:
            dec_output = dec_layer.forward(dec_output, enc_output, enc_output)

        # Pass the final decoder stack output to the linear layer that takes in
        # input vector of size 'embed_len' and outputs a vector that has the 
        # size of the vocab specified. Finall return the softmax output of that vector
        final_output = self.finalLinear(dec_output)

        return self.softmax(final_output)